In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
# 获取数据索引文件
train_df=pd.read_csv("/kaggle/input/ai-vs-human-generated-dataset/train.csv")
test_df=pd.read_csv("/kaggle/input/ai-vs-human-generated-dataset/test.csv")
# 随机划分验证集
train_df, valid_df = train_test_split(train_df, test_size=0.3, random_state=42)

In [2]:
from torchvision import transforms
# 定义数据预处理
transform_enhanced = transforms.Compose([
    transforms.Resize((512, 512)), 
    transforms.RandomCrop((224, 224)),  
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5), 
    transforms.RandomInvert(p=0.5),
    transforms.Resize((384, 384)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 通用 ImageNet 预训练均值和标准差
])
transform = transforms.Compose([
    transforms.Resize((384, 384)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 通用 ImageNet 预训练均值和标准差
])

In [3]:
import torch
from torch.utils.data import Dataset
import os
from PIL import Image
import random
# 定义设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 自定义数据集类
class TrainDataset(Dataset):
    def __init__(self,df):
        self.df=df
    def __getitem__(self,idx):
        img_path="/kaggle/input/ai-vs-human-generated-dataset/"
        img_path+=self.df.iloc[idx]['file_name']
        img=Image.open(img_path).convert("RGB")
        random_num=random.random()

        if random_num%2==0:
            img_tensor=transform_enhanced(img)
        else:
            img_tensor=transform(img)

        label=self.df.iloc[idx]['label']
        return img_tensor,label
        
    def __len__(self):
        return len(self.df)

class ValidDataset(Dataset):
    def __init__(self,df):
        self.df=df
    def __getitem__(self,idx):
        img_path="/kaggle/input/ai-vs-human-generated-dataset/"
        img_path+=self.df.iloc[idx]['file_name']
        img=Image.open(img_path).convert("RGB")
        img_tensor=transform(img)
        label=self.df.iloc[idx]['label']
        return img_tensor,label
    def __len__(self):
        return len(self.df)

class TestDataset(Dataset):
    def __init__(self,df):
        self.df=df
    def __getitem__(self,idx):
        img_path="/kaggle/input/ai-vs-human-generated-dataset/"
        img_path+=self.df.iloc[idx]['id']
        img=Image.open(img_path).convert("RGB")
        img_tensor=transform(img)
        return img_tensor
    def __len__(self):
        return len(self.df)

# 自定义数据集
train_dataset=TrainDataset(train_df)
valid_dataset=ValidDataset(valid_df)
test_dataset=TestDataset(test_df)

In [4]:
from torch.utils.data import DataLoader
# 定义加载器
train_dataloader=DataLoader(train_dataset,batch_size=16,shuffle=True)
valid_dataloader=DataLoader(valid_dataset,batch_size=16,shuffle=True)
test_dataloader=DataLoader(test_dataset,batch_size=16,shuffle=False)

In [5]:
import tqdm
import timm
from torch import nn
from torch.optim.lr_scheduler import StepLR

# 定义模型
model = timm.create_model('tf_efficientnetv2_s.in21k_ft_in1k', 
                          pretrained=True,
                         num_classes=2).to(device)

# 定义损失函数、优化器、调度器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=6e-6, weight_decay=1e-2)
scheduler = StepLR(optimizer, step_size=1, gamma=0.5)


model.safetensors:   0%|          | 0.00/86.5M [00:00<?, ?B/s]

In [6]:
from tqdm  import tqdm
def train(model,criterion,optimizer,scheduler,train_dataloader,valid_dataloader):
    total_loss = 0
    total=0
    tp=0
    fp=0
    fn=0
    model.train()
    s=0
    # 训练
    for batch,(tensor,label) in tqdm(enumerate(train_dataloader)):
        
        inputs=tensor.to(device)
        labels=label.to(device)
        total+=labels.size()[0]
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        predictions = torch.argmax(outputs, dim=1)
        for i in range(predictions.size()[0]):
            if(predictions[i].item() == labels[i].item() and labels[i].item()==1):
                tp+=1
            if(predictions[i].item() != labels[i].item() and labels[i].item()==1):
                fp+=1
            if(predictions[i].item() != labels[i].item() and labels[i].item()==0):
                fn+=1
        
        s+=1
        if(s%250==0):
            print(f"当前总精度为：{(total-fp-fn)/total:.4f}")
    
    p=tp/(tp+fp)
    r=tp/(tp+fn)
    f1=2*p*r/(p+r)
    # 验证
    model.eval()
    total_loss2 = 0
    total2=0
    tp2=0
    fp2=0
    fn2=0
    for batch,(tensor,label) in tqdm(enumerate(valid_dataloader)):
        
        inputs=tensor.to(device)
        labels=label.to(device)
        total2+=labels.size()[0]
        with torch.no_grad():
            outputs = model(inputs)
        loss = criterion(outputs, labels)

        total_loss2 += loss.item()
        predictions = torch.argmax(outputs, dim=1)
        for i in range(predictions.size()[0]):
            if(predictions[i].item() == labels[i].item() and labels[i].item()==1):
                tp2+=1
            if(predictions[i].item() != labels[i].item() and labels[i].item()==1):
                fp2+=1
            if(predictions[i].item() != labels[i].item() and labels[i].item()==0):
                fn2+=1
    p2=tp2/(tp2+fp2)
    r2=tp2/(tp2+fn2)
    f12=2*p2*r2/(p2+r2)
    scheduler.step()
    return (total-fp-fn)/total,f1,(total2-fp2-fn2)/total2,f12


def test(model,criterion,optimizer,scheduler,test_dataloader,epoch):
    output=pd.DataFrame({'id':[],'label':[]})
    index=0
    for batch,tensor in tqdm(enumerate(test_dataloader)):
        model.eval()
        inputs=tensor.to(device)
        with torch.no_grad():
            outputs = model(inputs)
        predictions = torch.argmax(outputs, dim=1)
        for i in range(predictions.size()[0]):
            t={'id':test_df.iloc[index]['id'],'label':predictions[i].item()}
            output.loc[index]=t
            index+=1
    output.to_csv(f"output_{epoch+1}epoch.csv",index=False)

In [7]:
for epoch in range(0,6):
    train_acc,train_f1,valid_acc,valid_f1=train(model,criterion,optimizer,scheduler,train_dataloader,valid_dataloader)

    print(f"第{epoch+1}轮训练总精度为：{train_acc:.4f}")
    print(f"第{epoch+1}轮训练f1score为：{train_f1:.4f}")

    print(f"第{epoch+1}轮验证总精度为：{valid_acc:.4f}")
    print(f"第{epoch+1}轮验证f1score为：{valid_f1:.4f}")
    
    test(model,criterion,optimizer,scheduler,test_dataloader,epoch)

250it [02:04,  2.04it/s]

当前总精度为：0.6552


500it [04:07,  2.05it/s]

当前总精度为：0.7370


750it [06:10,  2.05it/s]

...

3250it [23:35,  2.27it/s]

当前总精度为：0.9913


3498it [25:22,  2.30it/s]
1500it [05:27,  4.57it/s]


第6轮训练总精度为：0.9915
第6轮训练f1score为：0.9915
第6轮验证总精度为：0.9930
第6轮验证f1score为：0.9930


347it [02:54,  1.99it/s]
